# RAFT

In [1]:
! pip install -r ../requirements.txt

## Synthetic data generation phase

### Select the documents

In [2]:
import pandas as pd
#pd.set_option("display.max_colwidth", 0)

ds_name = "vampire-bat-small-DEMO"
doc_path = "../sample_data/vampire-bats/bats/Desmodus draculae - Wikipedia.pdf"
ds_path = f"dataset/{ds_name}"
print("Creating dataset: " + ds_name)

Creating dataset: vampire-bat-small-DEMO


### Clean up the DEMO folder

In [3]:
# Clean up demo folder only if it's a DEMO dataset
if ds_path.endswith("DEMO"):
    import shutil
    print(f"Cleaning demo folder {ds_path}")
    shutil.rmtree(ds_path, ignore_errors=True)
    print(f"Cleaning demo checkpoints folder {ds_path}")
    shutil.rmtree(ds_path + "-checkpoints", ignore_errors=True)

Cleaning demo folder dataset/vampire-bat-small-DEMO
Cleaning demo checkpoints folder dataset/vampire-bat-small-DEMO


### Generate Q/A/CoT fine-tuning dataset using RAFT from the domain specific documents

In [4]:
! python3 ../raft.py \
    --datapath "$doc_path" \
    --output $ds_path \
    --distractors 3 \
    --doctype pdf \
    --chunk_size 512 \
    --questions 1 \
    --workers 2 \
    --system-prompt-key llama \
    --completion_model Meta-Llama-3-70B-Instruct \
    --embedding_model text-embedding-ada-002

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
2024-05-21 02:14:22  INFO [  0%] raft Using checkpoint chunks /workspaces/gorilla/raft/azure-ai-studio-ft/dataset/vampire-bat-small-DEMO-checkpoints/chunks
2024-05-21 02:14:22  INFO [  0%] raft Retrieving chunks from ../sample_data/vampire-bats/bats/Desmodus draculae - Wikipedia.pdf of type pdf using the text-embedding-ada-002 model.
Chunking: 100%|██████████████████████| 1/1 [00:00<00:00,  1.09file/s, chunks=10]
2024-05-21 02:14:23  INFO [  0%] raft Using system prompt key llama
2024-05-21 02:14:23  INFO [  0%] raft Using 2 worker threads
Generating: 100%|█| 10/10 [01:10<00:00,  7.02s/chunk, last tok/s=2.22e+4, avg to
2024-05-21 02:15:33  INFO [  0%] raft Consumed 8531 prompt tokens, 2151 completion tokens, 10682 total tokens
Creating json from Arrow format: 100%|███████████| 1/1 [00:00<00:00, 226.55ba/s]
2024-05-21 02:15:33  IN

## Prepare training, validation and evaluation splits

In [5]:
raft_arrow_file = f"{ds_path}/data-00000-of-00001.arrow"
dataset_path = f"{ds_path}-files/{ds_name}-full.jsonl"
dataset_path_hf = f"{ds_path}-files/{ds_name}-hf.full.jsonl"

dataset_path_hf_train = f"{ds_path}-files/{ds_name}-hf.train.jsonl"
dataset_path_hf_valid = f"{ds_path}-files/{ds_name}-hf.valid.jsonl"
dataset_path_hf_eval = f"{ds_path}-files/{ds_name}-hf.eval.jsonl"

dataset_path_ft_train = f"{ds_path}-files/{ds_name}-ft.train.jsonl"
dataset_path_ft_valid = f"{ds_path}-files/{ds_name}-ft.valid.jsonl"
dataset_path_ft_eval = f"{ds_path}-files/{ds_name}-ft.eval.jsonl"

print(f"Reading arrow file {raft_arrow_file}")

Reading arrow file dataset/vampire-bat-small-DEMO/data-00000-of-00001.arrow


### Export arrow dataset to JSONL

In [6]:
! python ../format.py \
    --input $raft_arrow_file \
    --output $dataset_path_hf \
    --output-format hf

Generating train split: 10 examples [00:00, 2070.34 examples/s]
2024-05-21 02:15:36  INFO [    ] raft Dataset has 10 rows
2024-05-21 02:15:36  INFO [    ] raft Converting arrow file dataset/vampire-bat-small-DEMO/data-00000-of-00001.arrow to jsonl hf file dataset/vampire-bat-small-DEMO-files/vampire-bat-small-DEMO-hf.full.jsonl
Creating json from Arrow format: 100%|███████████| 1/1 [00:00<00:00, 124.34ba/s]


In [7]:
hf_full_df = pd.read_json(dataset_path_hf, lines=True)
hf_full_df.head(1)

,id,type,question,context,oracle_context,cot_answer,instruction
0,d7cc3cd2-6f03-4354-8996-b020f31690e3,general,What is the surname of the author mentioned?,{'sentences': [['Desmodus draculaeTemporal ran...,"Turvey, S.","To answer the question, we need to identify th...",<DOCUMENT>Desmodus draculaeTemporal range: Ple...


### Do the splitting

In [8]:
# split dataset into 80%/10%/10%
import numpy as np
samples_count = len(hf_full_df)
hf_train_df, hf_valid_df, hf_eval_df = np.split(hf_full_df, [int(.8*samples_count), int(.9*samples_count)])
hf_train_df.to_json(dataset_path_hf_train, orient="records", lines=True)
hf_valid_df.to_json(dataset_path_hf_valid, orient="records", lines=True)
hf_eval_df.to_json(dataset_path_hf_eval, orient="records", lines=True)

/workspaces/gorilla/raft/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


### Export training and validation datasets

In [9]:
! python ../format.py \
    --input $dataset_path_hf_train \
    --input-type jsonl \
    --output $dataset_path_ft_train \
    --output-format completion \
    --output-completion-prompt-column text\
    --output-completion-completion-column ground_truth

Generating train split: 8 examples [00:00, 1793.68 examples/s]
2024-05-21 02:15:38  INFO [    ] raft Dataset has 8 rows
2024-05-21 02:15:38  INFO [    ] raft Converting jsonl file dataset/vampire-bat-small-DEMO-files/vampire-bat-small-DEMO-hf.train.jsonl to jsonl completion file dataset/vampire-bat-small-DEMO-files/vampire-bat-small-DEMO-ft.train.jsonl
Filter out empty examples: 100%|█████████| 8/8 [00:00<00:00, 1776.02 examples/s]
Rename fields and add <STOP> token: 100%|█| 8/8 [00:00<00:00, 3266.59 examples/s
Creating json from Arrow format: 100%|███████████| 1/1 [00:00<00:00, 423.07ba/s]


In [10]:
! python ../format.py \
    --input $dataset_path_hf_valid \
    --input-type jsonl \
    --output $dataset_path_ft_valid \
    --output-format completion \
    --output-completion-prompt-column text\
    --output-completion-completion-column ground_truth

Generating train split: 1 examples [00:00, 228.67 examples/s]
2024-05-21 02:15:40  INFO [    ] raft Dataset has 1 rows
2024-05-21 02:15:40  INFO [    ] raft Converting jsonl file dataset/vampire-bat-small-DEMO-files/vampire-bat-small-DEMO-hf.valid.jsonl to jsonl completion file dataset/vampire-bat-small-DEMO-files/vampire-bat-small-DEMO-ft.valid.jsonl
Creating json from Arrow format: 100%|███████████| 1/1 [00:00<00:00, 436.36ba/s]


### Export evaluation datasets

In [11]:
! python ../format.py \
    --input $dataset_path_hf_eval \
    --input-type jsonl \
    --output $dataset_path_ft_eval \
    --output-format eval

Generating train split: 1 examples [00:00, 235.08 examples/s]
2024-05-21 02:15:42  INFO [    ] raft Dataset has 1 rows
2024-05-21 02:15:42  INFO [    ] raft Converting jsonl file dataset/vampire-bat-small-DEMO-files/vampire-bat-small-DEMO-hf.eval.jsonl to jsonl eval file dataset/vampire-bat-small-DEMO-files/vampire-bat-small-DEMO-ft.eval.jsonl
Creating json from Arrow format: 100%|███████████| 1/1 [00:00<00:00, 306.58ba/s]


In [12]:
pd.read_json(dataset_path_ft_eval, lines=True).head(2)

,question,gold_final_answer,context
0,What is the name of the indigenous people in B...,Mura people,"<DOCUMENT>Turvey, S.</DOCUMENT>\n<DOCUMENT>Arr..."
